In [1]:
! cd feature_extraction/require-solidity-parser/ && pip install .

Processing /cephyr/users/gabmor/Alvis/feature_extraction/require-solidity-parser
  Preparing metadata (setup.py) ... done
  Created wheel for solidity-parser: filename=solidity_parser-0.1.1-py3-none-any.whl size=57152 sha256=b896814ec0790f6572beb5aeaf0ea54074f11b34366120363766a1d8e6681653
  Stored in directory: /local/tmp.2439986/pip-ephem-wheel-cache-a52kvsxt/wheels/f2/1b/e7/10c8c557bac4dab1524f631eb91a3c72e1a5320050f4cbb02d
Successfully built solidity-parser
  Attempting uninstall: solidity-parser
    Found existing installation: solidity-parser 0.1.1
    Uninstalling solidity-parser-0.1.1:
      Successfully uninstalled solidity-parser-0.1.1

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [1]:
import os
os.environ.get('TMPDIR')

'/local/tmp.2455930'

In [2]:
from datasets import load_dataset

dataset = load_dataset("ASSERT-KTH/DISL", 'decomposed', num_proc=16)['train']
dataset

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Loading dataset shards:   0%|          | 0/17 [00:00<?, ?it/s]

Dataset({
    features: ['contract_name', 'file_path', 'contract_address', 'language', 'source_code', 'abi', 'compiler_version', 'optimization_used', 'runs', 'constructor_arguments', 'evm_version', 'library', 'license_type', 'proxy', 'implementation', 'swarm_source'],
    num_rows: 514506
})

In [3]:
index = list(range(len(dataset)))
dataset_idx = dataset.add_column('index', index)
dataset_idx

Dataset({
    features: ['contract_name', 'file_path', 'contract_address', 'language', 'source_code', 'abi', 'compiler_version', 'optimization_used', 'runs', 'constructor_arguments', 'evm_version', 'library', 'license_type', 'proxy', 'implementation', 'swarm_source', 'index'],
    num_rows: 514506
})

In [4]:
# remove Vyper files (check language column)
dataset_sol = dataset_idx.filter(lambda x: x['language'] != 'Vyper')
dataset_sol



Filter:   0%|          | 0/514506 [00:00<?, ? examples/s]

Dataset({
    features: ['contract_name', 'file_path', 'contract_address', 'language', 'source_code', 'abi', 'compiler_version', 'optimization_used', 'runs', 'constructor_arguments', 'evm_version', 'library', 'license_type', 'proxy', 'implementation', 'swarm_source', 'index'],
    num_rows: 513639
})

In [5]:
dataset_min = dataset_sol.remove_columns(['contract_name', 'file_path', 'contract_address', 'language', 'abi', 'compiler_version', 'optimization_used', 'runs', 'constructor_arguments', 'evm_version', 'library', 'license_type', 'proxy', 'implementation', 'swarm_source'])
dataset_min

Dataset({
    features: ['source_code', 'index'],
    num_rows: 513639
})

# Create labels for the dataset
For every source code call the change_require_statements_string funtion, this function will return a list of source codes with the require statements removed
The original source code will be the label ground truth and the source code without the require statements will be the input


In [7]:
import sys
from feature_extraction.infillAbstractContext import change_require_statements_string_with_comment

def create_labels(batch):
    infilled_batch = []
    source_codes = batch['source_code']
    indices = batch['index']
    
    for i in range(len(source_codes)):
        try:
            infillings = change_require_statements_string(source_codes[i], "require(<FILL_ME>);")
            for infill in infillings:
                infilled_batch.append({'input': infill[0], 'label': infill[1], 'original_idx': indices[i], 'predicate': infill[2], 'comment': infill[3]})
        except Exception as e: print(e)
    
    return {'data': infilled_batch }

sys.path.insert(0, '..')

dataset_labels = dataset_min.map(
    create_labels,
    batched=True,
    num_proc=64,
    batch_size=1000,
    remove_columns=['source_code', 'index']
)
dataset_labels

Map (num_proc=64):   0%|          | 0/513639 [00:00<?, ? examples/s]

line 26:59 no viable alternative at input '_}'
line 27:66 no viable alternative at input '_}'


_requireSystemCollaterized();
_requireSystemRemoveCollaterized();
_requireSystemCollaterized();
_requireSystemCollaterized();
_requireSystemCollaterized();
_requireSystemRemoveCollaterized();
_requireSystemRemoveCollaterized();


line 1:0 token recognition error at: '@'
line 1:3 mismatched input '.2' expecting 'constant'
line 1:0 token recognition error at: '@'
line 1:3 mismatched input '.2' expecting 'constant'


list index out of range


...

list index out of range


line 1:0 token recognition error at: '@'
line 1:3 mismatched input '.4' expecting 'constant'


_requireNotPaused();
_requirePaused();


line 1:0 token recognition error at: '@'
line 1:3 mismatched input '.9' expecting 'constant'
line 1:0 token recognition error at: '@'
line 1:3 mismatched input '.4' expecting 'constant'
line 1:0 token recognition error at: '@'
line 1:3 mismatched input '.5' expecting 'constant'
line 1:0 token recognition error at: '@'
line 1:3 mismatched input '.0' expecting 'constant'
line 1:0 token recognition error at: '@'
line 1:3 mismatched input '.0' expecting 'constant'
line 4:0 mismatched input '*' expecting {<EOF>, 'pragma', 'import', 'from', 'abstract', 'contract', 'interface', 'library', 'error', 'using', 'struct', 'function', 'event', 'enum', 'address', 'mapping', 'calldata', 'revert', 'var', 'bool', 'string', 'byte', 'callback', Int, Uint, Byte, Fixed, Ufixed, 'leave', 'payable', 'type', 'global', 'constructor', 'fallback', 'receive', Identifier}


Dataset({
    features: ['data'],
    num_rows: 5282156
})

In [8]:
dataset_flat = dataset_labels.flatten().rename_columns({'data.input': 'input', 'data.label': 'require', 'data.original_idx': 'original_idx', 'data.predicate': 'label', 'data.comment': 'comment'})
dataset_flat

Dataset({
    features: ['comment', 'input', 'label', 'original_idx', 'predicate'],
    num_rows: 5282156
})

In [21]:
print(dataset_flat[40]['input'])

/*
*/
// SPDX-License-Identifier: MIT
pragma solidity >=0.5.0 <0.8.0;
abstract contract Context {
    function _msgSender() internal view virtual returns (address payable) { 
    }
    function _msgData() internal view virtual returns (bytes memory) { 
    }
}
library Address {
    function isContract(address account) internal view returns (bool) { 
    }
    function sendValue(address payable recipient, uint256 amount) internal { 
    }
    function functionCall(address target, bytes memory data) internal returns (bytes memory) { 
    }
    function functionCall(address target, bytes memory data, string memory errorMessage) internal returns (bytes memory) { 
    }
    function functionCallWithValue(address target, bytes memory data, uint256 value) internal returns (bytes memory) { 
    }
    function functionCallWithValue(address target, bytes memory data, uint256 value, string memory errorMessage) internal returns (bytes memory) { 
    }
    function _functionCallWithValue(address ta

In [22]:
dataset_flat.push_to_hub('GGmorello/FLAMES_no_comment')

Uploading the dataset shards:   0%|          | 0/155 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/35 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/35 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/35 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/35 [00:00<?, ?ba/s]

...

Creating parquet from Arrow format:   0%|          | 0/35 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/35 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/35 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/35 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/GGmorello/FLAMES_no_comment/commit/b541f7117e4331cad11e8efaaf16c2a3e0aedd64', commit_message='Upload dataset (part 00003-of-00004)', commit_description='', oid='b541f7117e4331cad11e8efaaf16c2a3e0aedd64', pr_url=None, pr_revision=None, pr_num=None)

In [2]:
from datasets import load_dataset

dataset = load_dataset("GGmorello/FLAMES_no_comment", cache_dir=os.environ.get('TMPDIR'), num_proc=16)['train']


Resolving data files:   0%|          | 0/155 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/155 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/5282156 [00:00<?, ? examples/s]

Loading dataset shards:   0%|          | 0/157 [00:00<?, ?it/s]

In [3]:
import random

original_idx_values = list(set(dataset['original_idx']))
random.shuffle(original_idx_values)

train_split = int(len(original_idx_values) * 0.8)  # 80% train
val_split = int(len(original_idx_values) * 0.1)    # 10% validation
train_idx = original_idx_values[:train_split]
val_idx = original_idx_values[train_split:train_split+val_split]
test_idx = original_idx_values[train_split+val_split:]



In [4]:
train_dataset = dataset.filter(lambda ex: ex['original_idx'] in train_idx, num_proc = 64)
val_dataset = dataset.filter(lambda ex: ex['original_idx'] in val_idx, num_proc = 64)
test_dataset = dataset.filter(lambda ex: ex['original_idx'] in test_idx, num_proc = 64)


Filter (num_proc=64):   0%|          | 0/5282156 [00:00<?, ? examples/s]

Filter (num_proc=64):   0%|          | 0/5282156 [00:00<?, ? examples/s]

Filter (num_proc=64):   0%|          | 0/5282156 [00:00<?, ? examples/s]

In [5]:
from datasets import DatasetDict
d = DatasetDict()
d['train'] = train_dataset
d['val'] = val_dataset
d['test'] = test_dataset

d.push_to_hub('GGmorello/FLAMES_no_comment', data_dir='data/split')

Uploading the dataset shards:   0%|          | 0/124 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/35 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/35 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/35 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/35 [00:00<?, ?ba/s]

...

Creating parquet from Arrow format:   0%|          | 0/34 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/34 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/34 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/436 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/GGmorello/FLAMES_no_comment/commit/6215d19b0fa681cdd5fe9267ea22c37bcedf1db7', commit_message='Upload dataset (part 00003-of-00004)', commit_description='', oid='6215d19b0fa681cdd5fe9267ea22c37bcedf1db7', pr_url=None, pr_revision=None, pr_num=None)